## 🛠️ Mod5 Data Challenge 6: Segmentation & RFM


**Why this activity?**  
You will practice building **RFM segmentation** from real e-commerce transactions to answer: *Who are our best customers, who is at risk, and who should we win back?*

**Dataset:** UCI Online Retail II (Excel, 2009–2011). 

**Goals — You will be able to:**
1) Load and minimally prepare real transaction data for RFM.  
2) Compute **Recency, Frequency, Monetary** per customer.  
3) Score and label simple **RFM segments**.  
4) Interpret segments into **business actions**.

**Interview practice:**
- **Q1:** Give two concrete actions you’d take from an RFM analysis.  
- **Q2:** How would you adapt RFM for a product with no revenue (e.g., a music app)?



### 👩‍🏫 Instructor-Led Demo (20 minutes)

### Step 1 — Load the Online Retail II Excel and prepare fields

We will:
1) Read both sheets from the Excel file and concatenate.  
2) Parse dates.  
3) Drop rows with missing **CustomerID** (needed for RFM).  
4) Create **activity_date** alias from `InvoiceDate`.  
5) Create **line_total = Quantity * UnitPrice** (we’ll use positive line totals for Monetary).

**Speaker Notes:**  
We keep setup minimal. For **Monetary**, we’ll use **positive line totals only** to avoid refunds/returns making negative spend dominate segments. In production, you’d decide whether to net refunds or keep separate KPIs.



In [2]:
# import packages per usual 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta

In [6]:
#YOU'VE SEEN THIS CODE ALREADY :) 

# Path to your downloaded Excel file
DATA_PATH = '/Users/ayemaq/Desktop/marcy_lab/DA2025_Lectures/Mod5/DataChallenges/data/online_retail_II.xlsx'

# 1) Read both sheets and concatenate
sheets = ["Year 2009-2010", "Year 2010-2011"]
df_list = [pd.read_excel(DATA_PATH, sheet_name=s, engine="openpyxl") for s in sheets]
df = pd.concat(df_list, ignore_index=True)

# 2) Parse datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

# 3) Drop missing CustomerID (required to build RFM)
df = df.dropna(subset=["Customer ID"]).copy()
df["Customer ID"] = df["Customer ID"].astype(int)

# 4) Activity date alias
df["activity_date"] = df["InvoiceDate"]

# 5) Line total (Monetary basis uses positive spend only)
df["line_total"] = df["Quantity"] * df["Price"]
snapshot_date = df["InvoiceDate"].max().normalize() + pd.Timedelta(days=1)

df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,activity_date,line_total
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,2009-12-01 07:45:00,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2009-12-01 07:45:00,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2009-12-01 07:45:00,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,2009-12-01 07:45:00,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,2009-12-01 07:45:00,30.0


### Step 2 — Select a market and compute RFM
We will:  
1) Filter to `Country == "United Kingdom"`  
2) Compute:  
   - **Recency**: days since last invoice  
   - **Frequency**: # of unique invoices per customer  
   - **Monetary**: total revenue per customer (can be negative if net refunds)

**Speaker Notes:**  
Frequency as **unique invoices** avoids inflating frequent line items on the same order.


In [9]:
uk_filter = df['Country'] == 'United Kingdom'
df_uk = df[uk_filter].copy()

posi_transaction_filter = df_uk['line_total'] > 0
df_uk_posi = df_uk[posi_transaction_filter].copy()

rfm_uk = (
    df_uk.groupby("Customer ID")
          .agg(
              Recency=("InvoiceDate", lambda s: (snapshot_date - s.max()).days),
              Frequency=("Invoice", "nunique"),
              Monetary=("line_total", "sum")
          )
          .reset_index()
)

rfm_uk.head()


,Customer ID,Recency,Frequency,Monetary
0,12346,325,17,-64.68
1,12608,404,1,415.79
2,12745,486,2,723.85
3,12746,527,3,230.85
4,12747,2,32,9164.59


### Step 3 — Score R, F, M and label segments
We will:  
1) Score each metric into **quartiles** (1–4)  
   - **Recency**: lower is better → labels `[4,3,2,1]`  
   - **Frequency/Monetary**: higher is better → labels `[1,2,3,4]`  
2) Combine to an **RFM_Score** (e.g., 443)  
3) Map simple labels (Champions, Loyal, At‑Risk, Lost)

**Speaker Notes:**  
Label rules are business‑defined. Keep logic simple and transparent.


In [ ]:
# Avoid errors if too few unique values for qcut:
def safe_qcut(x, q, labels):
    try:
        return pd.qcut(x.rank(method="first"), q, labels=labels)
    except ValueError:
        # fallback: all middle score if not enough unique values
        return pd.Series([labels[len(labels)//2]] * len(x), index=x.index)

# Scores
r_scores = pd.qcut(rfm_uk["Recency"], 4, labels=[4,3,2,1])  # lower recency = better
f_scores = safe_qcut(rfm_uk["Frequency"], 4, labels=[1,2,3,4])
m_scores = safe_qcut(rfm_uk['Monetary'], 4, labels=[1,2,3,4])

rfm_uk["R_score"] = r_scores.astype(int)
rfm_uk["F_score"] = f_scores.astype(int)
rfm_uk["M_score"] = m_scores.astype(int)
rfm_uk["RFM_Score"] = rfm_uk["R_score"].astype(str) + rfm_uk["F_score"].astype(str) + rfm_uk["M_score"].astype(str)

# Simple labeler
def label_rfm(row):
    r,f,m = row["R_score"], row["F_score"], row["M_score"]
    if r>=3 and f>=3 and m>=3:
        return "Champions"
    if r>=3 and f>=2:
        return "Loyal"
    if r<=2 and f>=3 and m>=3:
        return "At-Risk Loyalists"
    if r<=2 and f<=2 and m<=2:
        return "Lost"
    return "Others"

rfm_uk["Segment"] = rfm_uk.apply(label_rfm, axis=1)

rfm_uk.head()

,Customer ID,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_Score,Segment
0,12346,325,17,-64.68,2,4,1,241,Others
1,12608,404,1,415.79,1,1,2,112,Lost
2,12745,486,2,723.85,1,1,2,112,Lost
3,12746,527,3,230.85,1,2,1,121,Lost
4,12747,2,32,9164.59,4,4,4,444,Champions


### Step 4 — Summarize segments & propose actions

We will:  
1) Count customers per segment  
2) Compute total Monetary by segment (optional)  
3) List **actions** per segment (e.g., loyalty, win‑back)

**Speaker Notes:**  
Tie segments to decisions (email, pricing, UX). Emphasize clarity over complexity.


In [12]:
seg_counts = rfm_uk["Segment"].value_counts().rename_axis("Segment").to_frame("Customers")
seg_value  = rfm_uk.groupby("Segment")["Monetary"].sum().to_frame("Total_Monetary").sort_values("Total_Monetary", ascending=False)

display(seg_counts)
display(seg_value)

# Example actions (as a dict, could be converted to a table)
segment_actions = {
    "Champions": "VIP perks, early access, referrals",
    "Loyal": "Tiered rewards, personalized recommendations",
    "At-Risk Loyalists": "Win-back offers, survey for friction",
    "Lost": "Exit survey, targeted reactivation",
    "Others": "Onboarding education, nudges"
}
segment_actions


,Customers
Segment,
Champions,1689
Lost,1654
Others,729
Loyal,712
At-Risk Loyalists,626


,Total_Monetary
Segment,
Champions,1.071414e+07
At-Risk Loyalists,1.697792e+06
Others,5.053692e+05
Loyal,4.904536e+05
Lost,3.986659e+05


{'Champions': 'VIP perks, early access, referrals',
 'Loyal': 'Tiered rewards, personalized recommendations',
 'At-Risk Loyalists': 'Win-back offers, survey for friction',
 'Lost': 'Exit survey, targeted reactivation',
 'Others': 'Onboarding education, nudges'}

### 👩‍🏫 Student Led (30 minutes) -- ANSWER KEY

### Step 1 — Use the original dataframe (`df`) above and Filter to Germany and compute RFM
 
1) Filter `Country == "Germany"`  
2) Compute Recency (days since last invoice), Frequency (# unique invoices), Monetary (sum of amount)

In [ ]:
# 1) Filter to Germany
de_filter = df['Country'] == 'Germany'
df_de = df[de_filter].copy()

# 2) Compute RFM
rfm_de = df_de.groupby('Customer ID').agg(
    Recency = ('InvoiceDate', lambda x: (snapshot_date - x.max()).days),
    Frequency=("Invoice", "nunique"),
    Monetary=("line_total", "sum")
)

rfm_de.head()


,Recency,Frequency,Monetary
Customer ID,,,
12426,194,3,1311.41
12427,11,5,1011.87
12468,143,10,3624.55
12471,2,129,37948.61
12472,30,37,16956.69


### Step 2 — Score R, F, M and label segments

1) Create R, F, M scores (quartiles)  
2) Combine into `RFM_Score`  
3) Label segments with simple, explainable rules


In [ ]:
# Helper for qcut with few uniques
def safe_qcut(x, q, labels):
    try:
        return pd.qcut(x.rank(method="first"), q, labels=labels)
    except ValueError:
        return pd.Series([labels[len(labels)//2]] * len(x), index=x.index)

# Scores (fill in)
r_scores = pd.qcut(rfm_de['Recency'], 4, labels = [4,3,2,1]) 
f_scores = safe_qcut(rfm_de['Frequency'], 4, labels=[1,2,3,4])
m_scores = safe_qcut(rfm_de['Monetary'], 4, labels=[1,2,3,4])


rfm_de["R_score"] = r_scores.astype(int)
rfm_de["F_score"] = f_scores.astype(int)
rfm_de["M_score"] = m_scores.astype(int) 
rfm_de["RFM_Score"] = rfm_de['R_score'].astype(str) + rfm_de['F_score'].astype(str) + rfm_de['M_score'].astype(str)

 
 #Feel free to change the labels if you like :)
def label_rfm(row):
    r, f, m = row["R_score"], row["F_score"], row["M_score"]
    if r>=3 and f>=3 and m>=3:
        return "Goats"
    if r>=3 and f>=2:
        return "Loyal"
    if r<=2 and f>=3 and m>=3:
        return "Danger-zone"
    if r<=2 and f<=2 and m<=2:
        return "Lost in the sauce"
    return "Others"

rfm_de["Segment"] = rfm_de.apply(label_rfm, axis=1) 
rfm_de.head(10)


,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_Score,Segment
Customer ID,,,,,,,,
12426,194,3,1311.41,1,2,2,122,Lost in the sauce
12427,11,5,1011.87,4,2,2,422,Loyal
12468,143,10,3624.55,2,3,3,233,Danger-zone
12471,2,129,37948.61,4,4,4,444,Goats
12472,30,37,16956.69,3,4,4,344,Goats
12473,29,9,2848.23,3,3,3,333,Goats
12474,17,54,11966.45,4,4,4,444,Goats
12475,53,2,1149.64,2,1,2,212,Lost in the sauce
12476,1,38,13715.62,4,4,4,444,Goats


### Step 3 — Summarize segments and propose actions

1) Count customers per segment  
2) Compute total Monetary per segment (optional)  
3) Suggest 1 concrete action per major segment for **Germany**


In [19]:
seg_counts_de = rfm_de["Segment"].value_counts().rename_axis("Segment").to_frame("Customers")
seg_value_de  = rfm_de.groupby("Segment")["Monetary"].sum().to_frame("Total_Monetary").sort_values("Total_Monetary", ascending=False)

display(seg_counts_de)
display(seg_value_de)

,Customers
Segment,
Goats,30
Lost in the sauce,25
Others,22
Loyal,17
Danger-zone,13


,Total_Monetary
Segment,
Goats,308521.940
Danger-zone,47878.830
Others,30311.471
Loyal,16759.260
Lost in the sauce,14517.060


## Wrap‑Up (10 mins): Turning RFM into Action

Students:  Be prepared to compare UKs vs. Germany's RFM/insights :)

- **Explainability:** RFM is transparent—every stakeholder can understand Recency, Frequency, Monetary.
- **Actionability:** Map segments to clear plays:
  - **Champions:** VIP perks, early access, referrals
  - **Loyal:** Tiered rewards, personalized bundles
  - **At‑Risk Loyalists:** Win‑back offers, friction survey
  - **Lost:** Lightweight reactivation or exit survey
- **Caveats:** Monetary can be negative if refunds dominate—decide whether to clamp at 0 for your use case.
- **Next steps:**  
  - Compare countries or channels;  
  - Try deciles (1–10) for finer splits;  
  - Track segment movement month‑over‑month.


1. Goats (30 customers, highest spending ~308K)
These are your MVPs: loyal and high-value customers. We can keep them happy by rewarding their loyalty.
- offer exclusive perks like early access to new products, personalized discounts/promotions emails. Also, collect feedback to understand what keeps them engaged so you can replicate that for other groups.

2. Danger-Zone (13 customers, 47K total spending)
These users spend a lot but haven’t been active recently. There’s strong potential to re-engage them before they churn.
- Send personalized campaigns, limited-time offers, or product recommendations based on their past purchases to bring them back.

3. Loyal (17 customers, 16K total spending)
Consistent buyers but lower spenders; they’re reliable but could grow into “Goats.”
- Encourage upselling or using bundle offers, targeted product suggestions and highlight referral programs to turn them into brand advocates. 

4. Lost in the Sauce (25 customers, 14K total spending)
They’ve disengaged almost completely.
- Consider a reactivation campaign, like a win-back email or survey asking what made them stop purchasing. If that doesn’t work, it might be time to stop investing marketing resources here.

5. Others (22 customers, 30K total spending)
A mixed group that doesn’t fit neatly into any category.
- Monitor their behavior over the next period to see if they move into higher or lower tiers. You could test targeted campaigns to see what motivates them.